In [15]:
def max_index(lst):
    """Returns the index of the maximum element in the given list"""
    max_val = max(lst)
    max_idx = lst.index(max_val)
    return max_idx

def choose(keys,values,model_l,my_dict,X_input):
    date_string=''
    i=max_index(values)
    model=model_l[i]
    print('The highest accuracy is', keys[i] )
    print('The value is ',values[i])
    print('Choosing model: ',model)
    X_today=X_input.values
    time=X_input.index[0]
    date_string = time.strftime('%Y-%m-%d %H:%M:%S')
    date_string =date_string[:10]
    if model==lr_model:
        
        #print(df['Close'][-1])
        #lr_model.fit()
        lr_pred = np.where(lr_model.predict(X_today) > 0.5, 1, 0)
        
        print('Input Date: ', date_string)
        print('---------')
        #print(logreg_pred)
        if lr_pred[0]>0:
            print('Close price will go up after', date_string)
        else:
            print('will go down after', date_string)
    else:
        #model.fit(X_train, y_train)
        model_pred = model.predict(X_today)
        
        if model_pred[0]>0:
            print('Close price will go up on the day after', date_string)
        else:
            print('Close price will go down on the day after', date_string)

In [17]:
last=-1
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima.model import ARIMA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D
from tensorflow.keras.optimizers import Adam

#year='5y'
year='3y'
# 从yfinance API获取股票、国债和黄金数据
tqqq = yf.Ticker('TQQQ')
tqqq_df = tqqq.history(period=year)
bond = yf.Ticker('^TNX')
bond_df = bond.history(period=year)
gold = yf.Ticker('GC=F')
gold_df = gold.history(period=year)
tesla = yf.Ticker('TSLA')
tesla_df = tesla.history(period=year)

# 计算五日均线
tqqq_df['5_day_avg'] = tqqq_df['Close'].rolling(window=5).mean()
bond_df['5_day_avg'] = bond_df['Close'].rolling(window=5).mean()
gold_df['5_day_avg'] = gold_df['Close'].rolling(window=5).mean()
tesla_df['5_day_avg'] = tesla_df['Close'].rolling(window=5).mean()

# 合并所有数据
df = pd.DataFrame(index=tqqq_df.index)
df['Close']=tqqq_df['Close']
df['Open'] = tqqq_df['Open']
df['High'] = tqqq_df['High']
df['Low'] = tqqq_df['Low']
df['Volume'] = tqqq_df['Volume']
df['TQQQ_5_day_avg'] = tqqq_df['5_day_avg']
df['Bond_5_day_avg'] = bond_df['5_day_avg']
df['Gold_5_day_avg'] = gold_df['5_day_avg']
df['Tesla_5_day_avg'] = tesla_df['5_day_avg']

# 去掉有缺失值的行
df.dropna(inplace=True)
o_df=df.copy()
#df=df.iloc[:-1]  # remove today.  should remove this line (if at night), just now because it is in the trading time
# used N-1 day for the end of sample data
df=df.iloc[:last] 
# 计算涨跌情况

df['Price_Change'] = np.where(df['Close'].diff() > 0, 1, 0)

# 用昨天的数据预测今天的收盘价涨跌情况
df_close=df[['Close','Open', 'High', 'Low', 'Volume', 'TQQQ_5_day_avg', 'Bond_5_day_avg', 'Gold_5_day_avg', 'Tesla_5_day_avg']].iloc[:-1]

o_X_df=o_df[['Open', 'Volume', 'TQQQ_5_day_avg', 'Bond_5_day_avg', 'Gold_5_day_avg', 'Tesla_5_day_avg']]


# train data
X_df=df[['Open', 'Volume', 'TQQQ_5_day_avg', 'Bond_5_day_avg', 'Gold_5_day_avg', 'Tesla_5_day_avg']].iloc[:-1]
X = df[['Open', 'Volume', 'TQQQ_5_day_avg', 'Bond_5_day_avg', 'Gold_5_day_avg', 'Tesla_5_day_avg']].iloc[:-1].values
y = df['Price_Change'].iloc[1:].values

today_Xdf=o_df[['Open', 'Volume', 'TQQQ_5_day_avg', 'Bond_5_day_avg', 'Gold_5_day_avg', 'Tesla_5_day_avg']].iloc[-1:,:]
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 线性回归模型
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_pred = np.where(lr_model.predict(X_test) > 0.5, 1, 0)

# 逻辑回归模型
logreg_model = LogisticRegression()
logreg_model.fit(X_train, y_train)
logreg_pred = logreg_model.predict(X_test)

# 随机森林模型
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
'''
# 时间序列模型 (ARIMA)
ts_model = ARIMA(y_train, order=(1, 0, 0))
ts_model_fit = ts_model.fit()
ts_pred = np.where(ts_model_fit.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1) > 0, 1, 0)
'''
# 计算准确率
lr_accuracy = accuracy_score(y_test, lr_pred)
logreg_accuracy = accuracy_score(y_test, logreg_pred)
rf_accuracy = accuracy_score(y_test, rf_pred)
#ts_accuracy = accuracy_score(y_test, ts_pred)


print("Linear Regression Accuracy: {:.2f}%".format(lr_accuracy * 100))
print("Random Forest Accuracy: {:.2f}%".format(rf_accuracy * 100))
#print("Time Series (ARIMA) Accuracy: {:.3f}%".format(ts_accuracy * 100))
print("Logistic Regression Accuracy: {:.3f}%".format(logreg_accuracy * 100))
print('--------')
#t_df=X_df.iloc[-1,:]
time=X_df.index[-1]

date_str = time.strftime('%Y-%m-%d %H:%M:%S')
date_str =date_str[:10]
print('Train Data Last Date: ', date_str)
print('--------')
keys=['lr_accuracy', 'rf_accuracy' ,'logreg_accuracy']
values=[lr_accuracy, rf_accuracy, logreg_accuracy]
model_l=[lr_model,rf_model,logreg_model]
name_model_l=['lr_model','rf_model','logreg_model']
my_dict = dict(zip(keys, model_l))
#X_input=X_df.iloc[-1:, :]
choose(keys,values,model_l,my_dict,today_Xdf)

today_Xdf

Linear Regression Accuracy: 68.00%
Random Forest Accuracy: 52.00%
Logistic Regression Accuracy: 59.333%
--------
Train Data Last Date:  2023-04-17
--------
The highest accuracy is lr_accuracy
The value is  0.68
Choosing model:  LinearRegression()
Input Date:  2023-04-19
---------
Close price will go up after 2023-04-19


,Open,Volume,TQQQ_5_day_avg,Bond_5_day_avg,Gold_5_day_avg,Tesla_5_day_avg
Date,,,,,,
2023-04-19 00:00:00-04:00,26.870001,93469781,27.516,3.5478,2010.5,184.567996


In [18]:
df

,Close,Open,High,Low,Volume,TQQQ_5_day_avg,Bond_5_day_avg,Gold_5_day_avg,Tesla_5_day_avg,Price_Change
Date,,,,,,,,,,
2020-04-24 00:00:00-04:00,16.097124,15.464120,16.171305,15.135253,158990800,15.415160,0.6050,1713.059985,47.946267,0
2020-04-27 00:00:00-04:00,16.411156,16.611442,16.712821,16.230649,152370800,15.504177,0.6110,1715.119995,48.644801,1
2020-04-28 00:00:00-04:00,15.523464,16.868601,16.871074,15.481429,193123600,15.768753,0.6188,1721.580005,49.743467,0
2020-04-29 00:00:00-04:00,17.138121,16.579297,17.400226,16.341918,173052000,16.105037,0.6204,1716.520020,50.655466,1
2020-04-30 00:00:00-04:00,17.251865,17.410118,17.424954,16.801838,190272400,16.484346,0.6222,1706.700000,51.672133,1
...,...,...,...,...,...,...,...,...,...,...
2023-04-12 00:00:00-04:00,26.129999,27.340000,27.490000,26.000000,174019300,26.932000,3.3690,2007.520020,184.483997,0
2023-04-13 00:00:00-04:00,27.629999,26.530001,27.740000,26.490000,123761500,27.080000,3.4020,2011.600024,184.559995,1
2023-04-14 00:00:00-04:00,27.469999,27.209999,27.930000,26.700001,142392400,27.088000,3.4488,2009.660010,184.547995,0
